In [ ]:
import cv2
import numpy as np
import os
from collections import Counter

# Define paths
objects_folder = '../Objects'
scenes_folder = '../Scenes'
object_keypoints_folder = 'object_keypoints'
scene_keypoints_folder = 'scene_keypoints'
results_file = 'results.txt'

# Create directories if they don't exist
os.makedirs(object_keypoints_folder, exist_ok=True)
os.makedirs(scene_keypoints_folder, exist_ok=True)

def preprocess_image(image_path):
    """
    Load and preprocess an image:
    - Load in grayscale
    - Resize to 400x400
    - Histogram equalization
    - Gaussian blur
    - Edge-based segmentation using Canny edge detection
    """
    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize image
    img = cv2.resize(img, (400, 400))
    # Histogram equalization
    img = cv2.equalizeHist(img)
    # Gaussian blur
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # Edge-based segmentation
    edges = cv2.Canny(img, 50, 150)
    return edges

def detect_sift_features(image):
    """
    Detect SIFT keypoints and descriptors in an image.
    """
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def save_keypoints_image(image, keypoints, save_path):
    """
    Draw keypoints on the image and save it.
    """
    img_with_keypoints = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(save_path, img_with_keypoints)

# Preprocess and detect features in object images
objects = {}
for filename in os.listdir(objects_folder):
    if filename.endswith('.jpg'):
        object_path = os.path.join(objects_folder, filename)
        preprocessed_obj = preprocess_image(object_path)
        keypoints_obj, descriptors_obj = detect_sift_features(preprocessed_obj)
        objects[filename] = {
            'image': preprocessed_obj,
            'keypoints': keypoints_obj,
            'descriptors': descriptors_obj
        }
        # Save keypoints image
        save_path = os.path.join(object_keypoints_folder, filename)
        save_keypoints_image(preprocessed_obj, keypoints_obj, save_path)

# Initialize results dictionary
results = {}

# Process each scene
for scene_index in range(1, 11):
    scene_name = f'S{scene_index}'
    detected_objects = []
    for view in ['front', 'left', 'right']:
        scene_filename = f'{scene_name}_{view}.jpg'
        scene_path = os.path.join(scenes_folder, scene_filename)
        if not os.path.exists(scene_path):
            continue
        # Preprocess scene image
        preprocessed_scene = preprocess_image(scene_path)
        # Detect SIFT features in scene
        keypoints_scene, descriptors_scene = detect_sift_features(preprocessed_scene)
        # Save keypoints image
        save_scene_path = os.path.join(scene_keypoints_folder, scene_filename)
        save_keypoints_image(preprocessed_scene, keypoints_scene, save_scene_path)
        # Initialize BFMatcher
        bf = cv2.BFMatcher()
        for obj_name, obj_data in objects.items():
            # Match descriptors
            matches = bf.knnMatch(obj_data['descriptors'], descriptors_scene, k=2)
            # Apply ratio test
            good_matches = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)
            # Proceed if enough good matches are found
            if len(good_matches) > 10:
                # Extract location of good matches
                src_pts = np.float32([obj_data['keypoints'][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                dst_pts = np.float32([keypoints_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                # Find homography
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                matchesMask = mask.ravel().tolist()
                inliers = sum(matchesMask)
                # Dynamic thresholding based on inliers
                if inliers > 0.1 * len(good_matches):
                    detected_objects.append(obj_name)
    # Combine detections across views
    object_counts = Counter(detected_objects)
    final_objects = [obj for obj, count in object_counts.items() if count >= 2]
    results[scene_name] = final_objects

# Write results to results.txt
with open(results_file, 'w') as f:
    for scene, objs in results.items():
        obj_list = ', '.join(objs) if objs else 'No objects detected'
        f.write(f'{scene}: {obj_list}\n')


Results saved to testing.txt
